In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

from google.colab import drive
drive.mount('/content/drive')

import os

import matplotlib.pyplot as plt
import folium

import numpy as np
import pandas as pd

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Algunas funciones útiles

In [ ]:
# Función para agregar capas de GEE a Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# ROIs = ROIs.to_crs(epsg=4326)

def get_boundaries(gdf):
  ## from shapefile it returns city boundary as a rectangle in different format
  #### polygon_geom: shapely polygon
  #### polygon: geopandas dataframe
  #### geometry: Earth Engine polygon
  bounds = gdf.total_bounds
  lon_point_list = [bounds[0], bounds[0], bounds[2], bounds[2]]
  lat_point_list = [bounds[1], bounds[3], bounds[3], bounds[1]]
  polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
  # crs = {'init': 'epsg:4326'}
  polygon = gpd.GeoDataFrame(index=[0], geometry=[polygon_geom]) 
  # polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom]) 
  geometry = ee.Geometry.Polygon(
          [[[bounds[0], bounds[1]],
            [bounds[0], bounds[3]],
            [bounds[2], bounds[3]],
            [bounds[2], bounds[1]]]])
  return polygon_geom, polygon, geometry

def from_SHP_geom_to_GEE_geom(gdf):
  '''
  Converts geometry in Geopandas DataFram row to GEE geometry. Only tested over
  simple polygons
  '''
  coords = eval(gdf.to_json())['features'][0]['geometry']['coordinates']
  return ee.Geometry.Polygon(coords)


## Exploramos por arriba la base de datos

In [ ]:
base_path = '/content/drive/My Drive/Humedales/ROIs para extraccion de datos opticos/'
filename = 'Poligonos_UPI4_EPSG4326_join20220509_2.shp'
ROIs = gpd.read_file(base_path + filename)
ROIs.head()

In [ ]:
### Si queremos ver todos los polígonos juntos o el cuadrado alrededor de un polígono
# polygon_geom, polygon, geometry = get_boundaries(ROIs.iloc[[0]])
polygon_geom, polygon, geometry = get_boundaries(ROIs.iloc[[0]])

In [ ]:
### Si queremos ver un polígono tal cual sale de la base de datos
i = 0
# geometry = from_SHP_geom_to_GEE_geom(ROIs.iloc[[i]])

In [ ]:
first = (ee.ImageCollection('COPERNICUS/S2_SR')
         .filterBounds(geometry)
         .filterDate('2019-05-26', '2019-05-28')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first()
         .clip(geometry))

# Define a map centered on southern Maine.
map_s2 = folium.Map(location=(ROIs.centroid[i].y, ROIs.centroid[i].x), zoom_start=18)

# Add the image layer to the map and display it.
map_s2.add_ee_layer(
    first, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, 'first')


for _, r in ROIs.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry'])#.simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    # folium.Popup(r['BoroName']).add_to(geo_j)
    geo_j.add_to(map_s2)
map_s2



## Extracción de estadísticos sobre polígonos

Definimos la colección de imágenes

In [ ]:
system = 'Sentinel-2'
if system == 'Sentinel-2':
  images_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

### Método 1 - Iterando sobre geometrías y fechas.

Muy lento, pero más versátil.

In [ ]:
for i_geom in range(len(ROIs)):
  print(f'Geometría {i_geom}')
  geometry = from_SHP_geom_to_GEE_geom(ROIs.iloc[[i_geom]])
  images_collection_geometry = (images_collection
                              .filterBounds(geometry)
                          .sort('system:time_start', True))
  

  collectionList = images_collection_geometry.toList(images_collection_geometry.size())
  collectionSize = images_collection_geometry.size().getInfo()


  for i_image in range(collectionSize):
    image = ee.Image(collectionList.get(i_image)).clip(geometry) 
    
    print(ee.Date(image.get('system:time_start')).format("yyyy-MM-dd").getInfo())
    
    meanDictionary = image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=geometry,
    scale=10,
    maxPixels=1e9
    )

    print(meanDictionary.getInfo())
    stdDictionary = image.reduceRegion(
    reducer=ee.Reducer.stdDev(),
    geometry=geometry,
    scale=10,
    maxPixels=1e9
    )

    print(stdDictionary.getInfo())

    if i_image > 5:
      break

  if i_geom > 0:
    break

### Métodos 2 - Usando Map 

Mucho más eficiente. De esta forma obtenemos los estadísticos sobre todas las imágenes mucho más rápido. Seguimos iterando sobre todas las geometrías.



In [ ]:
# Funciones auxiliares

def mean_reduce(image):
  reduced = image.reduceRegion(
              reducer=ee.Reducer.mean(),
              geometry=geometry,
              scale=10)
  return ee.Feature(None, reduced)

def std_reduce(image):
  reduced = image.reduceRegion(
              reducer=ee.Reducer.stdDev(),
              geometry=geometry,
              scale=10)
  return ee.Feature(None, reduced)


def count_reduce(image):
  reduced = image.reduceRegion(
              reducer=ee.Reducer.count(),
              geometry=geometry,
              scale=10)
  return ee.Feature(None, reduced)

# mean_ = images_collection_geometry.map(mean_reduce)
# std_ = images_collection_geometry.map(std_reduce)

In [ ]:
dict_results = {}
for i_geom in range(len(ROIs)):
  print(f'Geometría {i_geom}')
  # row = ROIs.iloc[i_geom]
  geometry = from_SHP_geom_to_GEE_geom(ROIs.iloc[[i_geom]])
  images_collection_geometry = (images_collection
                              .filterBounds(geometry)
                          .sort('system:time_start', True))
  

  # collectionList = images_collection_geometry.toList(images_collection_geometry.size())
  # collectionSize = images_collection_geometry.size().getInfo()

  dates = (ee.List(images_collection_geometry
      .aggregate_array('system:time_start'))
      .map(lambda time_start: 
          ee.Date(time_start).format('Y-MM-dd')
      ).getInfo())

  means = images_collection_geometry.map(mean_reduce)
  stdDevs = images_collection_geometry.map(std_reduce)
  counts = images_collection_geometry.map(count_reduce)


  means_df = pd.DataFrame([d['properties'] for d in means.getInfo()['features']])
  stdDevs_df = pd.DataFrame([d['properties'] for d in stdDevs.getInfo()['features']])
  counts_df = pd.DataFrame([d['properties'] for d in counts.getInfo()['features']])
  
  # display(means_df.head())
  # display(stdDevs_df.head())
  # display(counts_df.head())

  dict_results[i_geom] = {'dates': dates, 'means': means_df,
                          'stdDevs': stdDevs_df, 'counts': counts_df}
  # print(stdDevs.getInfo())
  # if i_geom > 3:
  #   break


Llevamos al DataFrame.

In [ ]:
df_results = []
for i_geom in range(len(ROIs)):
  print(f'Geometría {i_geom}')
  row = ROIs.iloc[i_geom]
  means_df = dict_results[i_geom]['means']
  stdDevs_df = dict_results[i_geom]['stdDevs']
  counts_df = dict_results[i_geom]['counts']
  dates = dict_results[i_geom]['dates']

  for i_date in range(len(dates)):
    mask_cloud_prob_mean = means_df.loc[i_date, 'MSK_CLDPRB']
    for band in means_df.columns:
      
      new_row = row.copy()
      new_row['fecha'] = means_df.loc[i_date, band]
      new_row['banda_nombre'] = band
      new_row['angulo_incidencia'] = np.nan
      new_row['valor_promedio'] = means_df.loc[i_date, band]
      new_row['valor_desvioest'] = stdDevs_df.loc[i_date, band]
      new_row['prob_promedio_nubes'] = mask_cloud_prob_mean
      new_row['fecha'] = dates[i_date]
      new_row['sensor'] = system
      new_row['tipo_sensor'] = 'Optico'
      new_row['tipo_escena'] = system
      
      df_results.append(new_row)



In [ ]:
# df_result = pd.DataFrame()
df_results = pd.DataFrame(df_results)
df_results.reset_index(drop = True, inplace = True)
df_results.drop(columns = ['geometry'], inplace = True)
df_results.head()

In [ ]:
df_results.to_csv('/content/drive/My Drive/Humedales/ROIs para extraccion de datos opticos/Poligonos_UPI4_EPSG4326_join20220509_2_S2_SR_HARMONIZED.csv')

In [ ]:
aux = df_results[(df_results.prob_promedio_nubes == 0) & (df_results.banda_nombre.isin(['B4', 'B8']))]
ndvi = (aux[aux.banda_nombre == 'B8'].valor_promedio - aux[aux.banda_nombre == 'B4'].valor_promedio)/(aux[aux.banda_nombre == 'B8'].valor_promedio + aux[aux.banda_nombre == 'B4'].valor_promedio)

In [ ]:
plt.plot(aux[aux.banda_nombre == 'B8'].fecha, ndvi)

In [ ]:
plt.figure(figsize = (12,4))
plt.plot(aux.fecha, aux.valor_promedio)